In [8]:
"""Run this notebook from the repo root."""

'Run this notebook from the repo root.'

In [9]:
import os

os.chdir(r"C:\Users\zvfrf\PycharmProjects\meta-automl-research")

In [10]:
from meta_automl.data_preparation.models_loaders import KnowledgeBaseModelsLoader
import pickle
from tqdm import tqdm

In [11]:
models_loader = KnowledgeBaseModelsLoader()

for split in ("all",):
    # DataFrame in the form (dataset_id, dataset_name)
    train_datasets = models_loader.parse_datasets(split)

    models_for_train = {}
    for dataset_name in train_datasets["dataset_name"]:
        dataset_models = models_loader.load(
            dataset_names=[dataset_name],  # load models just for this exact dataset.
            fitness_metric="logloss",  # must correspond to a metric name in a knowledge base.
        )
        models_for_train[dataset_name] = dataset_models

    dataset_root = f"./data/knowledge_base_v0_homogeneous_dataset_{split}"
    pipelines_subdir = os.path.join(dataset_root, "pipelines")
    metrics_subdir = os.path.join(dataset_root, "metrics")

    if not os.path.exists(dataset_root):
        os.mkdir(dataset_root)
    if not os.path.exists(pipelines_subdir):
        os.mkdir(pipelines_subdir)
    if not os.path.exists(metrics_subdir):
        os.mkdir(metrics_subdir)

    for dataset, data in tqdm(models_for_train.items()):
        for i, record in enumerate(data):
            predictor = record.predictor
            fitness_value = record.fitness.value
            predictor.save(os.path.join(pipelines_subdir, f"{dataset}_{i}_pipeline.json"))
            with open(os.path.join(metrics_subdir, f"{dataset}_{i}_pipeline.pickle"), "wb") as f:
                pickle.dump({"fitness": fitness_value}, f)

100%|██████████| 43/43 [00:00<00:00, 167.46it/s]
